In [1]:
import ROOT

Welcome to JupyROOT 6.24/06


In [7]:
def doFitAndCreateWorkspace(fin,channel_name,nameout):

    fout = ROOT.TFile(nameout,"recreate")
    # output as a RooWorkspace
    rws = ROOT.RooWorkspace("HHWWgg","HHWWgg")

    # observable variable
    mass = ROOT.RooRealVar("mgg_%s"%channel_name,"invariant mass",125,100,180)
    getattr(rws,"import")(mass)

    # background model
    alpha = ROOT.RooRealVar("alpha_%s"%channel_name,"#alpha",-0.05,-0.2,0.01);
    expo = ROOT.RooExponential("exp_%s"%channel_name,"exponential function",mass,alpha);

    mch = fin.Get("Continuum_Bkg")
    dhbkg = ROOT.RooDataHist("mc_%s"%(channel_name),"obs data",ROOT.RooArgList(mass),mch)
    expo.fitTo(dhbkg)
    
    # make a nice plot just to check 
    c = ROOT.TCanvas()
    pl = mass.frame()
    dhbkg.plotOn(pl)
    expo.plotOn(pl)
    pl.Draw()
    c.SaveAs("background_fit_%s.pdf"%channel_name)
    getattr(rws,"import")(expo)

    # fit gaussians in a certain range
    mass.setRange("r1",123,127)

    # single-Higgs backgrounds & signal we'll assume each is a Gaussian
    for s in ["GGHH","GGH","VBFH","VH","ttH","tHq"]:
        mh    = ROOT.RooRealVar("mh_%s_%s"%(s,channel_name),"mean in GeV",125,123,127)
        sigma1 = ROOT.RooRealVar("sigma1_%s_%s"%(s,channel_name),"sigma1 in GeV",2.0,0.5,6)
        sigma2 = ROOT.RooRealVar("sigma2_%s_%s"%(s,channel_name),"sigma2 in GeV",4.0,0.5,9)
        gauss1 = ROOT.RooGaussian("gauss1_%s_%s"%(s,channel_name),"f1(m|M_{H},#sigma)",mass,mh,sigma1);
        gauss2 = ROOT.RooGaussian("gauss2_%s_%s"%(s,channel_name),"f2(m|M_{H},#sigma)",mass,mh,sigma2);
        frac   = ROOT.RooRealVar("frac_%s_%s"%(s,channel_name),"sigma1 in GeV",0.6,0.,1)
        gauss  = ROOT.RooAddPdf("gauss_%s_%s"%(s,channel_name),"",ROOT.RooArgList(gauss1,gauss2),ROOT.RooArgList(frac))
        th1   = fin.Get(s)
        dh1   = ROOT.RooDataHist("rdh_%s"%(s),"rdh_%s"%(s),ROOT.RooArgList(mass),th1)
        gauss.fitTo(dh1,ROOT.RooFit.Range("fitrange"))

        print(f"Results peak -> {s}, channel -> {channel_name}")
        mh.Print()
        sigma1.Print()
        sigma2.Print()
        print("---------------------------------")
        
        getattr(rws,"import")(gauss)

        # plot to check
        c = ROOT.TCanvas()
        pla = mass.frame()
        dh1.plotOn(pla)
        gauss.plotOn(pla)
        pla.Draw()
        c.SaveAs("signal_mc_fit_%s_%s.pdf"%(s,channel_name))
    
    for s in ["GGHH","GGH","VBFH","VH","ttH","tHq"]:
       # must set these parameters to constant for the limit setting
       rws.var("mh_%s_%s"%(s,channel_name)).setConstant(1)
       rws.var("sigma1_%s_%s"%(s,channel_name)).setConstant(1)
       rws.var("sigma2_%s_%s"%(s,channel_name)).setConstant(1)
       rws.var("frac_%s_%s"%(s,channel_name)).setConstant(1)
        


    # and lets import the data obs too!
    datah = fin.Get("data_obs")
    dh1   = ROOT.RooDataHist("data_obs_%s"%(channel_name),"obs data",ROOT.RooArgList(mass),datah)
    getattr(rws,"import")(dh1)


    rws.Print()
    fout.WriteTObject(rws)
    fout.Close()

In [9]:
fIn = ROOT.TFile.Open("data/Inv_mass_gghasOneL_DNN_2_HL.root")
doFitAndCreateWorkspace(fIn,"gghasOneL_DNN_2_HL","roofit_Inv_mass_gghasOneL_DNN_2_HL.root")


Results peak -> GGHH, channel -> gghasOneL_DNN_2_HL
---------------------------------
Results peak -> GGH, channel -> gghasOneL_DNN_2_HL
---------------------------------
Results peak -> VBFH, channel -> gghasOneL_DNN_2_HL
---------------------------------
Results peak -> VH, channel -> gghasOneL_DNN_2_HL
---------------------------------
Results peak -> ttH, channel -> gghasOneL_DNN_2_HL
---------------------------------
Results peak -> tHq, channel -> gghasOneL_DNN_2_HL
---------------------------------

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#1] INFO:ObjectHandling -- RooWorkspace::import(HHWWgg) importing RooRealVar::mgg_gghasOneL_DNN_2_HL
[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(exp_gghasOneL_DNN_2_HL) WARNING: a likelihood fit is requested of what appears to be wei

Info in <TCanvas::Print>: pdf file background_fit_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_GGHH_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_GGH_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_VBFH_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_VH_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_ttH_gghasOneL_DNN_2_HL.pdf has been created
Info in <TCanvas::Print>: pdf file signal_mc_fit_tHq_gghasOneL_DNN_2_HL.pdf has been created
